# Registering and Converting model using ML-ModelCI Tutorial

This tutorial loads a pretrained resnet50 model from torchvision, then register this model into modelHub, the converter will automatically convert this PyTorch format model into 2 model : a Open Neural Network eXchange (ONNX) format model and a TorchScipt format model, then the ONNX format model will be mannually converted into TensorRT format.

## 1. Start the ModelCI service

Firstly, we should set some environmemnt variables, especially mongodb related variables.

In [1]:
# set environment variables 
%env MONGO_HOST=localhost
%env MONGO_PORT=27017
%env MONGO_USERNAME=modelci
%env MONGO_PASSWORD=modelci@2020
%env MONGO_DB=modelci
%env MONGO_AUTH_SOURCE=modelci
%env BACKEND_CORS_ORIGINS="http://localhost,http://localhost:3000,http://localhost:8080,https://localhost:3000,https://localhost:8080"
%env PROJECT_NAME=modelci
%env SECRET_KEY=2a6c03b9ca06cd8fc3cf506f0ba924cb735f15918d54758426fd7282366a5e19

env: MONGO_HOST=localhost
env: MONGO_PORT=27017
env: MONGO_USERNAME=modelci
env: MONGO_PASSWORD=modelci@2020
env: MONGO_DB=modelci
env: MONGO_AUTH_SOURCE=modelci
env: BACKEND_CORS_ORIGINS="http://localhost,http://localhost:3000,http://localhost:8080,https://localhost:3000,https://localhost:8080"
env: PROJECT_NAME=modelci
env: SECRET_KEY=2a6c03b9ca06cd8fc3cf506f0ba924cb735f15918d54758426fd7282366a5e19


Then start the modelci service by following command:

In [2]:
%%bash
modelci start

2020-11-22 22:06:58,240 - ml-modelci Docker Container Manager - INFO - Container name=mongo-48933 stared
2020-11-22 22:06:59,464 - ml-modelci Docker Container Manager - INFO - Container name=cadvisor-11415 started.
2020-11-22 22:07:01,955 - ml-modelci Docker Container Manager - INFO - Container name=dcgm-exporter-61182 started.
2020-11-22 22:07:03,020 - ml-modelci Docker Container Manager - INFO - gpu-metrics-exporter-40241 stared
2020-11-22 22:07:03,221 - modelci backend - INFO - Uvicorn server listening on 8000
OMP: Error #13: Assertion failure at z_Linux_util.cpp(2338).
OMP: Hint Please submit a bug report with this message, compile and run commands used, and machine configuration info including native compiler and operating system versions. Faster response will be obtained by including all program sources. For information on submitting this issue, please see http://www.intel.com/software/products/support/.


## 2. Register ResNet50 Model


Firstly, we load pre-trained resnet50 model from torchvision, you can refer to <https://pytorch.org/docs/stable/torchvision/models.html> for more examples of pretrained models.

In [3]:
# load resnet50 model from torchvision
from torchvision import models
model = models.resnet50(pretrained=True)

Then we register this model into ModelHub

In [4]:
from modelci.hub.manager import register_model
from modelci.types.bo import Framework, IOShape, ModelVersion,Engine
from modelci.types.trtis_objects import ModelInputFormat
# set model input and output formats
inputs = [IOShape([-1, 3, 224, 224], dtype=float, name='INPUT__0', format=ModelInputFormat.FORMAT_NCHW)]
outputs = [IOShape([-1, 1000], dtype=float, name='probs')]
# register the model
register_model(
    model,
    dataset='imagenet',
    acc=0.76,
    task='image classification',
    inputs=inputs,
    outputs=outputs,
    architecture='ResNet50',
    framework=Framework.PYTORCH,
    version=ModelVersion('2')
)

2020-11-22 22:08:08,373 - converter - INFO - Use cached model
2020-11-22 22:08:08,376 - converter - INFO - Use cached model


## 3. Retrieve Models
By default, Converter will automatically convert registered models into optimized formats,PyTorch model can be converted to TorchScipt and ONNX formats, so we can retrieve two models from ModelHub.

In [5]:
# search for model
from modelci.hub.manager import retrieve_model
retrieved_models = retrieve_model(
        architecture_name = 'ResNet50',
        framework = Framework.PYTORCH,
        version=ModelVersion('2')
)

In [6]:
retrieved_models

We can compare detatiled information of these two models

In [7]:
retrieved_models[0].__dict__

{'_id': '5fba70ca215155fe3ddf0215',
 'name': 'ResNet50',
 'framework': <Framework.PYTORCH: 1>,
 'engine': <Engine.TORCHSCRIPT: 2>,
 'version': <modelci.types.bo.model_objects.ModelVersion at 0x7f92bc623b10>,
 'dataset': 'imagenet',
 'acc': 0.76,
 'task': 'image classification',
 'inputs': [<modelci.types.bo.model_objects.IOShape at 0x7f92bc623910>],
 'outputs': [<modelci.types.bo.model_objects.IOShape at 0x7f92bc6237d0>],
 'weight': <modelci.types.bo.model_objects.Weight at 0x7f92bc623250>,
 'profile_result': None,
 'status': <Status.UNKNOWN: 0>,
 'creator': 'sherry',
 'create_time': datetime.datetime(2020, 11, 22, 22, 8, 7, 822000)}

In [8]:
retrieved_models[1].__dict__

{'_id': '5fba70cc215155fe3ddf039f',
 'name': 'ResNet50',
 'framework': <Framework.PYTORCH: 1>,
 'engine': <Engine.ONNX: 3>,
 'version': <modelci.types.bo.model_objects.ModelVersion at 0x7f92bc624610>,
 'dataset': 'imagenet',
 'acc': 0.76,
 'task': 'image classification',
 'inputs': [<modelci.types.bo.model_objects.IOShape at 0x7f92bc5a7c10>],
 'outputs': [<modelci.types.bo.model_objects.IOShape at 0x7f92bc5a7510>],
 'weight': <modelci.types.bo.model_objects.Weight at 0x7f93053c68d0>,
 'profile_result': None,
 'status': <Status.UNKNOWN: 0>,
 'creator': 'sherry',
 'create_time': datetime.datetime(2020, 11, 22, 22, 8, 7, 822000)}

## 4. Convert models
We can convert models mannually. 

You can refer to <https://github.com/cap-ntu/ML-Model-CI/blob/master/docs/tutorial/convert.md> for more details.

In the following example, we will convert ONNX model into TensorRT format.

In [9]:
from modelci.hub.converter import TRTConverter
from modelci.hub.utils import generate_path
from modelci.types.bo import IOShape

# set model input and output formats
inputs = [IOShape([-1, 3, 224, 224], dtype=float, name='INPUT__0', format=ModelInputFormat.FORMAT_NCHW)]
outputs = [IOShape([-1, 1000], dtype=float, name='probs')]

In [10]:
# get ONNX model saved path
onnx_path = retrieved_models[1].saved_path
onnx_path

PosixPath('/home/sherry/.modelci/ResNet50/pytorch-onnx/2.onnx')

In [11]:
# set TensorRT format model save path
save_path = generate_path(
    model_name='ResNet50', framework=Framework.PYTORCH,engine=Engine.TRT,version=2
)
save_path

PosixPath('/home/sherry/.modelci/ResNet50/pytorch-trt/2')

In [12]:
from modelci.hub.converter import PyTorchConverter
TRTConverter.from_onnx(onnx_path, save_path, inputs=inputs, outputs=outputs)

Loading ONNX file from path /home/sherry/.modelci/ResNet50/pytorch-onnx/2.onnx...


True

## 5.Stop the ModelCI service

In [13]:
%%bash
modelci stop

2020-11-22 22:09:32,839 - ml-modelci Docker Container Manager - INFO - Container name=gpu-metrics-exporter-40241 stopped.
2020-11-22 22:09:33,971 - ml-modelci Docker Container Manager - INFO - Container name=dcgm-exporter-61182 stopped.
2020-11-22 22:09:34,602 - ml-modelci Docker Container Manager - INFO - Container name=cadvisor-11415 stopped.
2020-11-22 22:09:35,436 - ml-modelci Docker Container Manager - INFO - Container name=mongo-48933 stopped.
2020-11-22 22:09:35,554 - modelci backend - WARNING - No process is listening on 8000


In [14]:
%%bash
# remove running docker containers
docker rm $(docker ps -a -q)

8e4a54f19bfa
6682db491467
731a16a827d1
23b276f57d53
